In [2]:
#Codigo hecho por Antonia Arias, fcfm

import pandas as pd
import math
ss = pd.read_excel('/Users/antoniaarias/Desktop/cosas opti/Pacientes.xlsx', sheet_name = [0,1,2])

#LEER DATOS DE EXCEL A PYTHON

Pt = ss[0] #sheet de paciente con tratamiento
Pe = ss[1] #sheet de paciente con cita especifica
Cm = ss[2] #sheet de centro medico

Nt = Pt.index.tolist() #lista con los indecies en orden de los pacientes con tratamiento

Ne = Pe.index.tolist() #lista con los indeces en orden de los pacientes con cita


#listas aux tienen el largo de los subconjuntos Ne y Nt, pero los indices de N. ejemplo N= [0, 1, 2, 3, 4, 5] Cm=[0], Nt =[0,1], Ne=[0,1, 2]
#auxNe = [3, 4, 5] auxNt=[1,2]
def auxNe(Ne, Nt):
    auxNe = []
    for i in Ne:
        auxNe.append(i + len(Nt) + 1)
    
    return auxNe


def auxNt(Nt):
    auxNt = []
    for i in Nt:
        auxNt.append(i +1) 
        
    return auxNt
        
auxNe = auxNe(Ne, Nt)
auxNt = auxNt(Nt)



#definimos las frecuencias de visitas para cada tipo de paciente
#frecuencia de tratamiento para el paciente i 
ft_i = {}
for i in auxNt:
    n = i - 1 #indice segun el sheet
    ft_i[i] = int(Pt.iloc[n,1])

#cantidad de citas tomadas para el paciente j, NO ES IMPORTANTE PARA ESTE ESTUDIO PUES 
#ESTAMOS EN EL CASO PARTICULAR EN EL QUE SOLO SE RESERVA UNA VEZ A LA SEMANA
fe_j = {}
for j in auxNe:
    n = j - len(Nt) - 1 #indice segun el sheet
    fe_j[j] = int(Pe.iloc[n,1])


    
#el dia de la semana donde el paciente j agendo
d_j = {}
for j in auxNe:
    n = j - len(Nt) - 1 #indice segun el sheet
    d_j[j] = Pe.iloc[n,2]
    if d_j[j] == 'Lunes':
        d_j[j] = 1
    if d_j[j] == 'Martes':
        d_j[j] = 2
    if d_j[j] == 'Miercoles':
        d_j[j] = 3
    if d_j[j] == 'Jueves':
        d_j[j] = 4
    if d_j[j] == 'Viernes':
        d_j[j] = 5
        

        
        
#Creamos un nueva lista ordenada coveniente N de indices {0} Nt U Ne, el 0 es el Centro medico
def crearN(Ne, Nt):
    N = [0]
    CopyNt = Nt
    for i in range(0, len(CopyNt)):
        CopyNt[i] = i + 1

    CopyNe = Ne
    for j in range(0, len(CopyNe)):
        CopyNe[j] = j + len(Nt) + 1
        
    N.extend(CopyNt)
    N.extend(CopyNe)
    
    return N


        
N = crearN(Ne, Nt)


#definimos las distancias euclidianas entre cada lugar del tramo
#Funcion segun el numero de paciente en N entrega una lista con su [longitud, latitud]
def corrCoor(Nt, i):
    #Caso de paciente con cita
    if i >= len(Nt) + 1:
        ne = i - len(Nt) -1
        long_i = float(Pe.iloc[ne, 4])
        lat_i = float(Pe.iloc[ne, 3])
    else:
        if i==0:
            #centro medico
            long_i = float(Cm.iloc[0, 2])
            lat_i = float(Cm.iloc[0, 1])
        else:
            nt = i -1
            long_i = float(Pt.iloc[nt, 3])
            lat_i = float(Pt.iloc[nt, 2])
    
    
    return [long_i, lat_i]
            
print(Pt.iloc[0, 3])
    
N = crearN(Ne, Nt) 


      
def Distancia(i,j):
    long_i = corrCoor(Nt, i)[0]*111.1
    long_j = corrCoor(Nt, j)[0]*111.1
    lat_i = corrCoor(Nt, i)[1]*111.1
    lat_j = corrCoor(Nt, j)[1]*111.1
    dif1 = pow(long_i - long_j, 2)
    dif2 = pow(lat_i - lat_j, 2)
    return math.sqrt(dif1 + dif2)
    

#Definimos las distancias entre los lugares, donde i y. j representan la ubicacion del paciente i y j,
# y de ser i=0 o j=0 esto representa la ubicacion del centro medico.
Vel = 0.83333 #k/hm... 50k/h son 0.833 k/min 
T_ij = {}        
for i in N:
    for j in N:
        T_ij[i,j] = Distancia(i,j)/Vel
        

-103.27569


In [2]:
from gurobipy import *
model = Model('PVRP')
#se modifica Ne y queda igual a N

K = {1, 2, 3, 4, 5} #dias en los que se antiende


Nt = Pt.index.tolist() #lista con los indecies en orden de los pacientes con tratamiento

Ne = Pe.index.tolist() #lista con los indeces en orden de los pacientes con cita

N = crearN(Ne, Nt)

#lista de todos los pacientes que parte en 0 como el Centro Medico
def auxN(N):
    copyN = N
    copyN.pop(0)
    return copyN

auxN = auxN(N)
#L es un cantidad grande arbitraria, para este caso la definimos como la maxima cantidad podible de 
#pacientes que se podrían visitar
L = len(N) - 1

N = crearN(Ne, Nt)

CotaS = 80
CotaI= 0.5
u_kj = {}
x_ki = {}
y_kij = {}
#Variable de Desicion x: se visita o no al paciente i en el dia k
for k in K:
    for i in N:
        x_ki[k, i] = model.addVar( vtype='B', lb=0,name='x_%s,%s' %(k,i))


model.update()

#Variable de Desicion y: se hace o no el tramo de paciente i a j el dia k
for k in K:
    for i in N:
        for j in N:
            y_kij[k,i,j]= model.addVar( vtype='B', lb=0, name='y_%s,%s,%s' %(k,i,j))
            
#Variable de Desicion: u_kj, variable continua que indica el orden en el que el funsionario visito al paciente i en el dia k
for k in K:
    for j in auxN: #definir quizas en auxN
        u_kj[k,j] = model.addVar( vtype='C', lb=1, ub= 185,name='u_%s,%s,' %(k,j))
            
model.update()


    





    
#restriccion sub tour: Caso M la cantidad total de una cierta posicion sea menor o igual a las salidas desde 0
for k in K:
    for i in auxN:
        for j in auxN:
            model.addConstr(u_kj[k,j] + L*(1 - y_kij[k,i,j]) >= u_kj[k,i] + 1)

                            




        
        
#Restriccion 1, y_kij implica x_ki: si esta vale uno entonce x tambien. Si y_kij[k,i,j] vale 0 entonces
#R1.1: se acota por arriba con una cota mayor arbitraria
for k in K:
    for i in N:
        model.addConstr(x_ki[k,i] <= CotaS*quicksum(y_kij[k,i,j] for j in N))
model.update()
#R1.2: se acota por abajo con una cota menor arbitraria
for k in K:
    for i in N:
        model.addConstr(CotaI*quicksum(y_kij[k,i,j] for j in N) <= x_ki[k,i])
model.update()

#Restriccion 2, Salidas y Legadas:
#R2.1: para un dia k, puede salir a lo mas 1 funcionario desde una ubicacion i.
for k in K:
    for i in N:
        model.addConstr(quicksum(y_kij[k, i, j] for j in N)<=1)
model.update()


#R.2: para un dia k, puede llegar a lo mas 1 funcionario a la ubicacion j.
for k in K:
    for j in N:
        model.addConstr(quicksum(y_kij[k, i, j] for i in N)<=1)
model.update()

#Restriccion 3: se deben visitar a los pacientes en la frecuencia semanal que corresponde
#R3.1: caso pacientes con cita

print(auxNe)
for j in auxNe:
    model.addConstr(quicksum(x_ki[k, j] for k in K) == fe_j[j])
model.update()
    
#R3.2: caso pacientes con tratamiento
for i in auxNt:
    model.addConstr(quicksum(x_ki[k, i] for k in K) == ft_i[i])
    
model.update()


#Restriccion 4: a los pacientes con citas se les debe visitar el dia que corresponde      
for j in auxNe:
    d = d_j[j] # dia de la semana que agendo
    model.addConstr(quicksum(x_ki[k,j]*k for k in K) ==d)
    
          
model.update()

#Restriccion 5: el C.M es el punto de salida y llegada
#R 5.1: para un k fijo, debe haber exactamente una salida desde el C.M.
for k in K:
    model.addConstr(quicksum(y_kij[k,0,j] for j in N) == 1) #el uno se cambia por la cantidad de enfermeros en el otro caso

            
model.update()



#Restriccion6:La misma cantidad de salidas que llegadas
for k in K:
    for i in N:
        model.addConstr(quicksum(y_kij[k,i,j] for j in N) == quicksum(y_kij[k,l,i] for l in N))#o bien ambos valen 1 o ambos 0

#Restriccion 7: no hay viajes que salgan desde el lugar i y lleguen al mismo i.
for k in K:
    for i in auxN:
        model.addConstr(y_kij[k, i, i]==0)
        
model.update()


                            
#Restriccion 8:no reversible
for k in K:
    for i in N:
        for j in N:
            model.addConstr(y_kij[k, i,j]*y_kij[k,j,i]==0)  #por lo menos uno de estos valores es igual a 0    


            
model.update()           
#model.update()    
#tiempos en atender
s=10
p=20
                

#tiempo total semanal que se demora en atender a los pacientes S+P cste
P = quicksum(x_ki[k,i] for i in auxNt for k in K)*p
S = quicksum(x_ki[k,j] for j in auxNe for k in K)*s




                   
model.setObjective((quicksum(T_ij[i,j]*y_kij[k,i,j] for i in N for j in N for k in K) +S +P) , GRB.MINIMIZE)

model.Params.TIME_LIMIT=300.0

model.update()

model.optimize()


print('El tiempo minimo es', model.ObjVal, 'minutos totales esa semana')



#funciones de prueba para visualizar el ruteo
#Mostrar la ruta recorrida cada dia
rut = [0]

#funcion devuelve una lista con los pacientes visitados en orden y al final esta la cantidad de total de 
#lugares distintos visitados
def rutak(k, rut):
    i = rut[len(rut)-1]
    for j in N:
        if y_kij[k,i,j].x ==1:
            if j!=0:
                rut.append(j)
                rutak(k, rut)
                print(rut)
            else:
                rut.append(j)
                rut.append(len(rut)-1)
                return rut
            

        
        
                
def quienk(k):
    for i in N:
        if x_ki[k,i].x==1:
            print(i)
            
def adonde(k):
    for i in N:
        for j in N:
            if y_kij[k,i,j].x==1:
                print(i,j)
            


def asistencia(k):
    total = quicksum(x_ki[k,i].x for i in N)
        
    return total
        

            
#print(asistencia(1))
rutak(1, rut)
quienk(1)
adonde(1)




Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-16
[41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185]
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 177035 rows, 174835 columns and 1552910 no

In [ ]:
from gurobipy import *
mo= Model('PVRPconM')

#se modifica Ne y queda igual a N

K = {1, 2, 3, 4, 5} #dias en los que se antiende



Nt = Pt.index.tolist() #lista con los indecies en orden de los pacientes con tratamiento
L= len(Nt) - 1
Ne = Pe.index.tolist() #lista con los indeces en orden de los pacientes con cita

N = crearN(Ne, Nt)


def auxN(N):
    copyN = N
    copyN.pop(0)
    return copyN

auxN = auxN(N)

N = crearN(Ne, Nt)

#Cntidad de funcionarios trabajando todos los dias
M={1}

def auxN(N):
    copyN = N
    copyN.pop(0)
    return copyN

auxN = auxN(N)

N = crearN(Ne, Nt)


#cotas arbitrarias menor y mayor a 1
CotaS = 80
CotaI= 0.5
u_mkj = {}
x_mki = {}
y_mkij = {}
#Variable de Desicion x: si es que el funcionario m se visita o no al paciente i en el dia k.
for k in K:
    for m in M:
        for i in N:
            x_mki[m, k, i] = mo.addVar( vtype='B', lb=0,name='x_%s,%s,%s' %(m,k,i))


mo.update()

#Variable de Desicion y: si el funcionario m hace o no el tramo de paciente i a j el dia k.
for k in K:
    for m in M:
        for i in N:
            for j in N:
                y_mkij[m,k,i,j]= mo.addVar( vtype='B', lb=0, name='y_%s,%s,%s,%s' %(m,k,i,j))
            
            
mo.update()


#Variable de Desicion:
for k in K:
    for m in M:
        for j in auxN: #definir quizas en auxN
            u_mkj[m,k,j] = mo.addVar( vtype='C', lb=1, ub= 185,name='u_%s,%s,%s' %(m,k,j))
            
mo.update()


    
    
#Restricciones: variable u
#u_0 =1




    
#si yij =1 y u_i=1 => u_j =1 +1
  
#restriccion sub tour: Caso M la cantidad total de una cierta posicion sea menor o igual a las salidas desde 0
for k in K:
    for m in M:
        for i in auxN:
            for j in auxN:
                mo.addConstr(u_mkj[m,k,j] + L*(1 - y_mkij[m,k,i,j]) >= u_mkj[m,k,i] + 1)
mo.update()



          



#Restriccion 1, y_kij implica x_ki: si y_kij vale 1 entonce x_ kij tambien. Si y_kij vale 0 entonces
#x_ki tambien
#R1.1: se acota por arriba con una cota mayor arbitraria
for k in K:
    for m in M:
        for i in N:
            mo.addConstr(x_mki[m,k,i] <= CotaS*quicksum(y_mkij[m,k,i,j] for j in N))
mo.update()
#R1.2: se acota por abajo con una cota menor arbitraria
for k in K:
    for m in M:
        for i in N:
            mo.addConstr(CotaI*quicksum(y_mkij[m,k,i,j] for j in N) <= x_mki[m,k,i])
mo.update()

#Restriccion 2, Salidas y Legadas:
#R2.1: para un dia k fijo, puede salir a lo mas 1 funcionario desde una ubicacion i. Con i distinto de 0.
for k in K:
    for i in auxN:
        mo.addConstr(quicksum(y_mkij[m,k, i, j] for j in N for m in M)<=1)#auxN
mo.update()




#Restriccion 3: se deben visitar a los pacientes en la frecuencia semanal que corresponde
#R3.1: caso pacientes con cita
for j in auxNe:
    mo.addConstr(quicksum(x_mki[m,k,j] for k in K for m in M) == fe_j[j])
mo.update()
    
#R3.2: caso pacientes con tratamiento
for i in auxNt:
    mo.addConstr(quicksum(x_mki[m,k, i] for k in K for m in M) == ft_i[i])
    
mo.update()


#Restriccion 4: a los pacientes con citas se les debe visitar el dia que corresponde      
for j in auxNe:
    mo.addConstr(quicksum(x_mki[m,k,j]*k for k in K for m in M) == d_j[j])
    
          
mo.update()

#Restriccion 5: el C.M es el punto de salida y llegada
#R 5.1: para un k fijo, debe haber a lo mas M salidas desde el C.M.
for k in K:
    for m in M:
        mo.addConstr(quicksum(y_mkij[m,k,0,j] for j in N) == 1) #quizas estricto ==
            
mo.update()




#Restriccion 6: Misma cantidad de salidas que de llegadas
for k in K:
    for m in M:
        for i in N:
            mo.addConstr(quicksum(y_mkij[m,k,i,j] for j in auxN) == quicksum(y_mkij[m,k,l,i] for l in auxN))#o bien ambos valen 1 o ambos 0

mo.update()
#Restriccion 7: no hay viajes que salgan desde el lugar i y lleguen al mismo i.
for k in K:
    for m in M:
        for i in N:
            mo.addConstr(y_mkij[m, k, i, i]==0)
        
mo.update()


#Restriccion 7:El tramo no es reversible.
for k in K:
    for m in M:
        for i in N:
            for j in N:
                mo.addConstr(y_mkij[m, k, i,j]*y_mkij[m,k,j,i]==0)  #por lo menos uno de estos valores es igual a 0    

            


            

                
mo.update()    
#tiempos en atender
s=10
p=20
                

    
P = quicksum(x_mki[m,k,i] for i in auxNt for k in K for m in M)*p
S = quicksum(x_mki[m,k,j] for j in auxNe for k in K for m in M)*s




                   
mo.setObjective((quicksum(T_ij[i,j]*y_mkij[m,k,i,j] for i in N for j in N for k in K for m in M) +S +P) , GRB.MINIMIZE)

mo.Params.TIME_LIMIT=300.0

mo.update()

mo.optimize()


print('El tiempo minimo es', mo.ObjVal, 'minutos totales esa semana')

        

In [ ]:
#función que entrega una lista con los pacientes que se atienden en el dia k
def Aten(k):
    A=[]
    for i in N:
        if x_ki[k,i].x==1:
            A=A+[i]
    return A
#fuencion que entrega el "paciente" que se visita despues del i-paciente
def dire(k,i):
    for j in Aten(k):
        if y_kij[k,i,j].x ==1:
            return j
assert dire(1,0)==113
#función que entrega la ruta del dia k
def ruta(k):
    p=0
    A=[0]
    while p<len(Aten(k)):
        i=dire(k,A[p])
        A=A+[i]
        p=p+1
    return A
#ahora se grafica cada ruta de cada dia
for k in K:
    A=ruta(k)
    la=[]
    lo=[]
    for i in A:
        VV=corrCoor(Nt,i)
        la=la+[VV[1]]
        lo=lo+[VV[0]]
    plt.scatter(lo, la)
    plt.plot(lo,la,color="green",label="Ruta del dia k")   
    plt.xlabel("Latitud")
    plt.ylabel("Longitud")
    plt.legend()
    plt.grid()
    plt.show()
    print(ruta(k))